# データ準備とYOLOv8用データセット構築

このノートブックは、YOLOv8モデルのトレーニングに使用するデータセットを準備するプロセスをガイドします。以下のステップを実行します。

1.  **動画からのフレーム抽出**: 指定されたディレクトリ内の動画ファイルからフレームを抽出します。
2.  **アノテーションの準備**: 抽出されたフレームにアノテーションを付けるための準備とガイダンスを提供します。
3.  **データセットの分割**: アノテーション済みの画像とラベルを、訓練セット、検証セット、およびオプションでテストセットに分割します。
4.  **`data.yaml` の生成**: YOLOv8がトレーニングに必要なデータセット構成ファイル (`data.yaml`) を生成します。


## 1. ライブラリのインポート

まず、必要なPythonライブラリをインポートします。

In [ ]:
import os
import cv2
import numpy as np
import subprocess
import shutil
from pathlib import Path
import random
import yaml
from tqdm import tqdm

print("ライブラリが正常にインポートされました。")

## 2. 設定

データ処理に必要なパスとパラメータを設定します。

- `RAW_VIDEO_DIR`: フレームを抽出する元の動画ファイルが格納されているディレクトリ。
- `DATASET_BASE_DIR`: 処理済みデータセット（抽出されたフレーム、アノテーションファイル、分割済みデータセット、`data.yaml`）が保存されるベースディレクトリ。
- `FRAME_WIDTH`, `FRAME_HEIGHT`: 抽出するフレームの幅と高さ。YOLOv8の入力サイズに合わせて調整します。
- `FRAME_RATE`: 動画から抽出するフレームのレート（1秒あたりのフレーム数）。`None` の場合は動画のオリジナルFPSを使用。
- `CLASS_NAMES`: データセット内のクラス名リスト。`data.yaml` に記述されます。
- `TRAIN_RATIO`, `VAL_RATIO`: データセットを訓練セットと検証セットに分割する際の比率。残りがテストセットになります。

In [ ]:
# --- 設定項目 ---

# 入力動画ディレクトリ (例: '../data/raw_videos/')
RAW_VIDEO_DIR = '../data/raw/' 

# データセットのベースディレクトリ (例: '../data/yolo_dataset/')
# このディレクトリ以下に images, labels, train, val, test フォルダが作成されます。
DATASET_BASE_DIR = '../data/processed/yolo_prepared_dataset/'

# フレームサイズ
FRAME_WIDTH = 640
FRAME_HEIGHT = 480

# フレームレート (None の場合、動画のオリジナルFPSを使用)
# 特定のFPSで抽出したい場合は数値を指定 (例: 10)
FRAME_RATE_EXTRACTION = None 

# クラス名 (data.yaml用) - プロジェクトに合わせて変更してください
CLASS_NAMES = ['player', 'ball', 'net'] # 例

# データ分割比率
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
# TEST_RATIO は (1.0 - TRAIN_RATIO - VAL_RATIO) で自動計算されます

# --- 設定項目ここまで ---

# パスオブジェクトの作成
raw_video_path = Path(RAW_VIDEO_DIR)
dataset_base_path = Path(DATASET_BASE_DIR)
extracted_images_path = dataset_base_path / 'images' # 抽出された全フレームを一時的に保存
extracted_labels_path = dataset_base_path / 'labels' # アノテーションファイルを保存する場所

# 必要なディレクトリを作成
extracted_images_path.mkdir(parents=True, exist_ok=True)
extracted_labels_path.mkdir(parents=True, exist_ok=True) # アノテーション用

print(f"入力動画ディレクトリ: {raw_video_path.resolve()}")
print(f"データセットベースディレクトリ: {dataset_base_path.resolve()}")
print(f"抽出フレーム保存先: {extracted_images_path.resolve()}")
print(f"アノテーションファイル保存先 (期待される場所): {extracted_labels_path.resolve()}")
print(f"フレームサイズ: {FRAME_WIDTH}x{FRAME_HEIGHT}")
if FRAME_RATE_EXTRACTION:
    print(f"フレーム抽出レート: {FRAME_RATE_EXTRACTION} fps")
else:
    print("フレーム抽出レート: オリジナル動画のFPSを使用")
print(f"クラス名: {CLASS_NAMES}")
print(f"訓練セット比率: {TRAIN_RATIO}, 検証セット比率: {VAL_RATIO}")

## 3. 動画からのフレーム抽出

指定された `RAW_VIDEO_DIR` 内のすべての動画ファイル（.mp4, .avi, .mov, .mkv）からフレームを抽出します。
抽出されたフレームはリサイズされ、`DATASET_BASE_DIR/images/` ディレクトリに `videoName_frame_XXXXXX.png` の形式で保存されます。

**注意:** `ffmpeg` がシステムにインストールされ、PATHが通っている必要があります。

In [ ]:
def extract_frames_from_videos(video_dir, output_image_dir, width, height, fps=None):
    """
    動画ディレクトリからフレームを抽出し、指定されたサイズにリサイズして保存する。

    Args:
        video_dir (Path): 動画ファイルが格納されているディレクトリ。
        output_image_dir (Path): 抽出されたフレームを保存するディレクトリ。
        width (int): フレームの幅。
        height (int): フレームの高さ。
        fps (int, optional): 抽出するフレームレート。Noneの場合はオリジナルFPS。
    """
    video_files = [f for f in video_dir.iterdir() if f.is_file() and f.suffix.lower() in ['.mp4', '.avi', '.mov', '.mkv']]
    
    if not video_files:
        print(f"{video_dir} に動画ファイルが見つかりませんでした。")
        return

    print(f"{len(video_files)} 本の動画ファイルを処理します...")

    for video_file in tqdm(video_files, desc="動画処理中"):
        video_name = video_file.stem
        
        # ffmpeg コマンドの構築
        command = [
            'ffmpeg',
            '-i', str(video_file),
            '-vf', f'scale={width}:{height}' + (f',fps={fps}' if fps else ''),
            '-qscale:v', '2',  # 高画質でフレームを抽出
            str(output_image_dir / f'{video_name}_frame_%06d.png')
        ]
        
        try:
            # ffmpegコマンドの標準出力と標準エラー出力を抑制する
            process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout, stderr = process.communicate()
            if process.returncode != 0:
                print(f"動画 {video_file.name} の処理中にffmpegエラーが発生しました。")
                print(f"コマンド: {' '.join(command)}")
                print(f"エラー出力: {stderr.decode('utf-8', errors='ignore')}")
                continue # 次の動画へ
            # print(f"動画 {video_file.name} からフレームを抽出しました。")
        except FileNotFoundError:
            print("エラー: ffmpeg コマンドが見つかりません。ffmpegがインストールされ、PATHが通っていることを確認してください。")
            return
        except Exception as e:
            print(f"動画 {video_file.name} の処理中に予期せぬエラーが発生しました: {e}")
            continue

    print(f"全動画のフレーム抽出が完了しました。フレームは {output_image_dir} に保存されています。")

# フレーム抽出の実行
extract_frames_from_videos(raw_video_path, extracted_images_path, FRAME_WIDTH, FRAME_HEIGHT, FRAME_RATE_EXTRACTION)

## 4. フレームのアノテーション

フレームの抽出が完了したら、次にこれらのフレームにアノテーションを付ける必要があります。
アノテーションは、画像内のオブジェクトの位置とクラスを定義する作業です。

**アノテーションツール:**
[LabelImg](https://github.com/HumanSignal/labelImg), [CVAT](https://github.com/cvat-ai/cvat), [Label Studio](https://labelstud.io/) などのアノテーションツールを使用できます。
これらのツールは、YOLO形式でのアノテーションエクスポートをサポートしています。

**YOLOアノテーション形式:**
各画像に対して、同じ名前（拡張子を除く）の `.txt` ファイルを作成します。例えば、`my_video_frame_000001.png` のアノテーションは `my_video_frame_000001.txt` になります。
各 `.txt` ファイルには、画像内のオブジェクトごとに1行の記述が含まれます。
各行の形式は次のとおりです: `<class_id> <x_center_norm> <y_center_norm> <width_norm> <height_norm>`
- `<class_id>`: オブジェクトのクラスを示す整数（0から始まるインデックス）。これは `CLASS_NAMES` リストのインデックスに対応します。
- `<x_center_norm>`, `<y_center_norm>`: オブジェクトのバウンディングボックスの中心のx座標とy座標（画像の幅と高さで正規化された値、0から1の範囲）。
- `<width_norm>`, `<height_norm>`: オブジェクトのバウンディングボックスの幅と高さ（画像の幅と高さで正規化された値、0から1の範囲）。

**手順:**
1.  上記で設定した `extracted_images_path` (`{DATASET_BASE_DIR}/images/`) にあるフレームに対してアノテーションを行います。
2.  生成された `.txt` アノテーションファイルを、`extracted_labels_path` (`{DATASET_BASE_DIR}/labels/`) に保存します。
    各画像ファイル（例: `frame_001.png`）に対して、対応するラベルファイル（例: `frame_001.txt`）が同じディレクトリ構造で保存されるようにしてください。

**このノートブックの次のセルに進む前に、すべてのアノテーション作業を完了し、ラベルファイルを正しい場所に保存してください。**

In [ ]:
# アノテーションが完了したことを確認するための簡単なチェック
# extracted_labels_path に .txt ファイルが存在するかどうかを確認します。

label_files = [f for f in extracted_labels_path.iterdir() if f.is_file() and f.suffix.lower() == '.txt']
image_files_for_check = [f for f in extracted_images_path.iterdir() if f.is_file() and f.suffix.lower() in ['.png', '.jpg', '.jpeg']]

if not image_files_for_check:
    print(f"警告: {extracted_images_path} に画像ファイルが見つかりません。フレーム抽出が正しく行われたか確認してください。")
elif not label_files:
    print(f"警告: {extracted_labels_path} にアノテーションファイル (.txt) が見つかりません。")
    print("アノテーション作業を完了し、ラベルファイルを上記のディレクトリに保存してください。")
else:
    print(f"{extracted_labels_path} に {len(label_files)} 個のアノテーションファイルが見つかりました。")
    # 画像ファイルとラベルファイルの対応関係を一部チェック (オプション)
    missing_labels = 0
    for img_f in image_files_for_check[:min(len(image_files_for_check), 5)] : # 最初の5件でチェック
        expected_label_f = extracted_labels_path / (img_f.stem + ".txt")
        if not expected_label_f.exists():
            missing_labels +=1
            print(f"  - 画像 {img_f.name} に対応するラベルファイル {expected_label_f.name} が見つかりません。")
    if missing_labels == 0 and image_files_for_check:
         print("基本的な画像とラベルのペアリングは問題なさそうです（最初の数件で確認）。")

print(f"
アノテーションが完了したら、次の「データセットの分割と data.yaml の生成」セルを実行してください。")

## 5. データセットの分割と `data.yaml` の生成

アノテーション済みの画像とラベルを、訓練 (train)、検証 (val)、およびオプションでテスト (test) セットに分割します。
その後、YOLOv8がデータセットの場所とクラス情報を知るために必要な `data.yaml` ファイルを生成します。

In [ ]:
def split_and_organize_dataset(base_dir, images_source_dir, labels_source_dir, 
                               train_ratio=0.7, val_ratio=0.15, class_names=None, data_yaml_filename='data.yaml'):
    """
    画像とラベルを train/val/test セットに分割し、YOLO形式のディレクトリ構造を構築し、data.yaml を生成する。
    """
    base_dir = Path(base_dir)
    images_source_dir = Path(images_source_dir)
    labels_source_dir = Path(labels_source_dir)

    if class_names is None:
        class_names = ['object'] # デフォルト

    # 新しいディレクトリ構造のパスを定義
    train_images_dir = base_dir / 'train' / 'images'
    train_labels_dir = base_dir / 'train' / 'labels'
    val_images_dir = base_dir / 'val' / 'images'
    val_labels_dir = base_dir / 'val' / 'labels'
    test_images_dir = base_dir / 'test' / 'images'
    test_labels_dir = base_dir / 'test' / 'labels'

    # ディレクトリを作成
    for d in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir, test_images_dir, test_labels_dir]:
        d.mkdir(parents=True, exist_ok=True)

    # 画像ファイルを取得
    image_files = [f for f in images_source_dir.iterdir() if f.is_file() and f.suffix.lower() in ['.png', '.jpg', '.jpeg']]
    if not image_files:
        print(f"エラー: {images_source_dir} に画像ファイルが見つかりません。")
        return

    dataset_pairs = []
    for img_path in image_files:
        label_filename = img_path.stem + '.txt'
        label_path = labels_source_dir / label_filename
        if label_path.exists():
            dataset_pairs.append((img_path, label_path))
        else:
            print(f"警告: 画像 {img_path.name} に対応するラベルファイル {label_path.name} が {labels_source_dir} に見つかりません。スキップします。")

    if not dataset_pairs:
        print("エラー: 有効な画像とラベルのペアが見つかりませんでした。")
        return

    random.shuffle(dataset_pairs)
    total_pairs = len(dataset_pairs)
    train_end_idx = int(total_pairs * train_ratio)
    val_end_idx = train_end_idx + int(total_pairs * val_ratio)

    train_pairs = dataset_pairs[:train_end_idx]
    val_pairs = dataset_pairs[train_end_idx:val_end_idx]
    test_pairs = dataset_pairs[val_end_idx:]

    def move_pairs(pairs, target_img_dir, target_lbl_dir, set_name):
        if not pairs:
            print(f"{set_name} セットにはファイルがありません。")
            return
        print(f"{len(pairs)} ペアを {set_name} セットに移動中...")
        for img_path, lbl_path in tqdm(pairs, desc=f"{set_name} セット移動"):
            try:
                shutil.copy(str(img_path), str(target_img_dir / img_path.name)) # copyの代わりにmoveも可
                shutil.copy(str(lbl_path), str(target_lbl_dir / lbl_path.name)) # copyの代わりにmoveも可
            except Exception as e:
                print(f"ファイル移動エラー ({img_path.name} または {lbl_path.name}): {e}")
    
    move_pairs(train_pairs, train_images_dir, train_labels_dir, "訓練")
    move_pairs(val_pairs, val_images_dir, val_labels_dir, "検証")
    move_pairs(test_pairs, test_images_dir, test_labels_dir, "テスト")
    
    # data.yaml の作成
    data_yaml_path = base_dir / data_yaml_filename
    
    # base_dir からの相対パスで記述
    # YOLOv8の ultralytics ライブラリは、data.yaml 内のパスを data.yaml ファイル自身の場所からの相対パスとして解釈することが多い。
    # そのため、'../train/images' のような形式ではなく、data.yaml が dataset_base_dir にある場合、
    # 'train/images' のように記述する。
    # `path` ディレクティブは data.yaml の親ディレクトリを指すように設定する。
    
    data_config = {
        'path': str(base_dir.resolve()), # データセットのルートパス (絶対パスを推奨)
        'train': str((train_images_dir).relative_to(base_dir)), #'train/images', 
        'val': str((val_images_dir).relative_to(base_dir)), #'val/images',
        'nc': len(class_names),
        'names': class_names
    }
    if test_pairs: # テストセットが実際に作成された場合
        data_config['test'] = str((test_images_dir).relative_to(base_dir)) #'test/images'
    
    try:
        with open(data_yaml_path, 'w') as f:
            yaml.dump(data_config, f, sort_keys=False, default_flow_style=None)
        print(f"{data_yaml_path} を作成/更新しました。")
        print("--- data.yaml の内容 ---")
        print(yaml.dump(data_config, sort_keys=False, default_flow_style=None))
        print("------------------------")
    except Exception as e:
        print(f"エラー: {data_yaml_path} の書き込みに失敗しました: {e}")

    print("データセットの分割と data.yaml の生成が完了しました。")
    print(f"訓練データ: {train_images_dir}, {train_labels_dir}")
    print(f"検証データ: {val_images_dir}, {val_labels_dir}")
    if test_pairs:
        print(f"テストデータ: {test_images_dir}, {test_labels_dir}")
    else:
        print("テストセットは作成されませんでした。")

# データセット分割と data.yaml 生成の実行
# extracted_images_path と extracted_labels_path は、アノテーション済みの全画像と全ラベルが格納されている場所
split_and_organize_dataset(
    dataset_base_path, 
    extracted_images_path, 
    extracted_labels_path,
    train_ratio=TRAIN_RATIO,
    val_ratio=VAL_RATIO,
    class_names=CLASS_NAMES
)

## 6. まとめと次のステップ

このノートブックでは、以下の処理を行いました。
1.  元の動画からフレームを抽出し、指定されたサイズにリサイズして `{DATASET_BASE_DIR}/images/` に保存しました。
2.  抽出されたフレームに対するアノテーションの実施方法と、アノテーションファイル (`.txt`) を `{DATASET_BASE_DIR}/labels/` に保存するよう指示しました。
3.  アノテーション済みの画像とラベルを、訓練・検証・テストセットに分割し、YOLOv8が要求するディレクトリ構造 (`{DATASET_BASE_DIR}/train/`, `{DATASET_BASE_DIR}/val/`, `{DATASET_BASE_DIR}/test/`) に整理しました。
4.  YOLOv8のトレーニングに必要な `data.yaml` ファイルを `{DATASET_BASE_DIR}/` に生成しました。

**次のステップ:**
これで、YOLOv8モデルのトレーニングを開始する準備が整いました。
生成された `data.yaml` ファイルのパスをトレーニングスクリプトまたはノートブックに指定して、モデルのトレーニングを実行してください。

**生成されたディレクトリ構造の確認:**
`{DATASET_BASE_DIR}` ディレクトリを開き、以下の構造になっていることを確認してください。
```
{DATASET_BASE_DIR}/
├── data.yaml
├── images/              # (オプション: 元の全抽出フレーム、分割後は空でも良い)
│   ├── video1_frame_000001.png
│   └── ...
├── labels/              # (オプション: 元の全アノテーション、分割後は空でも良い)
│   ├── video1_frame_000001.txt
│   └── ...
├── train/
│   ├── images/
│   │   └── ... (訓練用画像)
│   └── labels/
│       └── ... (訓練用ラベル)
├── val/
│   ├── images/
│   │   └── ... (検証用画像)
│   └── labels/
│       └── ... (検証用ラベル)
└── test/  (オプション)
    ├── images/
    │   └── ... (テスト用画像)
    └── labels/
        └── ... (テスト用ラベル)
```